In [10]:
import numpy
import random
from operator import add
import matplotlib.pyplot as plt
from igraph import *

In [43]:
def edge_selector(neighbor_tracking):
    checker = 'NaN'
    while checker == 'NaN':
        val = random.choice(neighbor_tracking.keys())
        checker = neighbor_tracking[val]
    return val

In [44]:
def Mover(position_selected, layout_old, illegal_moves):
    possibilities = {1: (1, 0), 2: (0,1), 3:(-1, 0), 4:(0,-1), 5:(0,0)}
    movement_options = 4
    
    newDir = random.choice(possibilities.values())
    new_posit = tuple(map(add, position_selected, newDir))
    
    new_position_selected = boundary_checker(position_selected, spaceMin, spaceMax)
    
    if new_position_selected == position_selected:
    
        if new_posit not in layout_old:        
            idx = layout_old.get(position_selected)

            del layout_old[position_selected]
            layout_old[new_posit] = idx

            return layout_old, new_posit
        else:
            return layout_old, position_selected
    else:
        if new_position_selected not in layout_old:        
            idx = layout_old.get(position_selected)

            del layout_old[position_selected]
            layout_old[new_position_selected] = idx

            return layout_old, new_position_selected
        else:
            return layout_old, position_selected

In [45]:
def illegal_position_generator(spaceMax, spaceMin):
    from itertools import permutations
    boundary = spaceMax + 2
    lower_boundary = spaceMin - 1
    items = range(lower_boundary, boundary)
    moves = {}
    i = 0
    for p in permutations(items, 2):
        moves[i] = p
        i += 1

    illegal_moves = {}
    legal_moves = {}
    for key,value in moves.items():
        for n,i in enumerate(value):
            try:        
                if i <= spaceMin or i >= spaceMax:
                    illegal_moves[key] = value
            except:
                continue
                
    for key, value in illegal_moves.iteritems():
        if value in moves.values():
            del moves[key]

    #This changes values into keys and vice versa
    moves = {y:x for x,y in moves.iteritems()}
    illegal_moves = {y:x for x,y in illegal_moves.iteritems()}
   
    # include positions at edges
    illegal_moves.update({(spaceMin,spaceMin):10000000, (spaceMax, spaceMax):100000001 })
    
    return moves, illegal_moves

def layout_gen(node_number, legal_moves): 
    import random
    
    wanted_keys = random.sample(legal_moves.keys(), node_number)
    #using dictionary comprehension to subset a dictionary by keys
    layout_old = dict((k, legal_moves[k]) for k in wanted_keys if k in legal_moves)
    
    
    return layout_old#, layout_names

In [46]:
def allRxn(move, bind, dissociate):

    options = numpy.zeros((3, 1))
    options[0] = move/(move + bind + dissociate)
    options[1] = (bind + move)/(move + bind + dissociate)
    options[2] = (bind + move + dissociate)/(move + bind + dissociate)
    
    random_value = random.uniform(0, 1)
    
    if 0 <= random_value <= options[0]:
        return 'move'
    elif options[0] <= random_value <= options[1]:
        return 'bind'
    else:
        return 'dissociate'

In [47]:
def boundary_checker(x, spaceMin, spaceMax):
    if spaceMax in x or spaceMin in x:
        x = list(x)
        if x[0] == spaceMin:
            x[0] = spaceMin + 1
        elif x[0] == spaceMax:
            x[0] = spaceMax - 1
        elif x[1] == spaceMin:
            x[1] = spaceMin + 1
        elif x[1] == spaceMax:
            x[1] = spaceMax - 1
    return tuple(x)

In [48]:
from operator import add

position_selected = (8, 5)
spaceMin = 0
spaceMax = 9

def movement_finder(position_selected, illegal_moves, layout_old, spaceMin, spaceMax):
    total_moves = 4
    my_possibilities = {1: (1, 0), 2: (0,1), 3:(-1, 0), 4:(0,-1) }

    finding_neighbors = {}
    j = 0
    for i in my_possibilities.values():
        finding_neighbors[tuple(map(add, position_selected, i))] = j
        j += 1

   
    if position_selected[0] == spaceMin and position_selected[1] == spaceMin:
        current_moves = 2
        sd1 = set(finding_neighbors.keys())
        sd2 = set(layout_old.keys())
        current_moves = current_moves - len(sd1.intersection(sd2))
        return current_moves
    if position_selected[0] == spaceMax and position_selected[1] == spaceMax:
        current_moves = 2
        sd1 = set(finding_neighbors.keys())
        sd2 = set(layout_old.keys())
        current_moves = current_moves - len(sd1.intersection(sd2))
        return current_moves 
    if position_selected[0] == spaceMin or position_selected[1] == spaceMin:
        current_moves = 3
        sd1 = set(finding_neighbors.keys())
        sd2 = set(layout_old.keys())
        current_moves = current_moves - len(sd1.intersection(sd2))
        return current_moves
    if position_selected[0] == spaceMax or position_selected[1] == spaceMax:
        current_moves = 3
        sd1 = set(finding_neighbors.keys())
        sd2 = set(layout_old.keys())
        current_moves = current_moves - len(sd1.intersection(sd2))
        return current_moves
    else:
        sd1 = set(finding_neighbors.keys())
        sd2 = set(layout_old.keys())
        #sd3 = set(illegal_moves.keys())
        current_moves = total_moves - len(sd1.intersection(sd2)) #- len(sd1.intersection(sd3))
        return current_moves

## New Association Functiom

In [49]:
g = Graph(len(layout_old))
g.vs['name'] = layout_old.keys()
new_dict = {}
def new_association_function(x, d, g, new_dict, val):
    new_list = []
    posit = numpy.array([[1,0], [0,1], [-1, 0], [0,-1] ])
    
    new_posit = posit + x
    for i in new_posit:
        b = tuple(i)
        if b in d:
            new_list.append(b)
            new_dict[x] = new_list
    if x in new_dict and val == True:
        options = new_dict[x]
        toBind = random.choice(options)
        v1_position = g.vs['name'].index( str(toBind) )
        v2_position = g.vs['name'].index( str(x) )
        if g.degree(v1_position) < 4 and g.degree(v2_position) < 4:
            g.add_edge(v1_position, v2_position)
        return new_dict[x]
    elif x in new_dict and val == False:
        return new_dict[x]
    else:
        new_dict[x] = 'NaN'
        return new_dict[x]    

# Gillespie

## Initiate node layout

In [50]:
spaceMax = 60
spaceMin = 0
node_number = 3000
legal_moves, illegal_moves = illegal_position_generator(spaceMax, spaceMin)
layout_old = layout_gen(node_number, legal_moves)

## This initiates the movement count

In [51]:
move_count = {}

for k,v in layout_old.items():
    val = movement_finder(k, illegal_moves, layout_old, spaceMin, spaceMax)
    move_count[k] = val

## This initiates the neighbor tracking

In [52]:
import collections
g = Graph(len(layout_old))

output = {}
i = 0
for k,v in layout_old.items():
    output[str(k)] = i
#    output[k] = i
    i += 1    
od = collections.OrderedDict(sorted(output.items()))
g.vs['name'] = od.keys()


neighbor_tracking = {}

for k,v in layout_old.items():
    new_association_function(k, layout_old, g, neighbor_tracking, False)
    
    

## This initiates neighbor edge count

In [53]:
bind_count = {}
for k,v in neighbor_tracking.items():
    bind_count[k] = len(v)

In [54]:
t = 0
total_time = 5000
k_move = 1e-3
k_bind = 5e-5
k_off = 0.001
tau = 1e-3
times = []
degree_dist = {}
degree_dist_all = {}

# this sets up initial movement options
#for k,v in layout_old.items():
#    movement_positions[v] = movement_finder(k, illegal_moves, layout_old, spaceMin, spaceMax)
        
while t <= total_time:


    ## Calculate all possible reactions

    move = sum(move_count.values() )                              # movement
    move_k = move * k_move * tau

    bind = sum(bind_count.values() )                              # binding
    bind_k = bind * k_bind * tau

    dissociate = len(g.get_edgelist())                            # dissociation
    dissociate_k = dissociate * k_off * tau


    rate_sum = move_k + bind_k + dissociate_k                     # Time at which the next reaction will take place
    total_tau = random.expovariate(lambd=rate_sum)                      # Time at which the next reaction will take place

    if not times:
        times.append(total_tau)
    else:
        total_tau = total_tau + times[t-1]
        times.append(total_tau)                                             # Save time as a tuple
        
    reaction = allRxn(move_k, bind_k, dissociate_k)               # Sample from uniform distribution

    if reaction == 'move':
        position_selected = random.choice(layout_old.keys() )                  # randomly select a node
        #print 'move'
        #print 'this is position_selected 1:', position_selected
        
        if position_selected in layout_old:
            #print 'position select is in layout_old'
            del move_count[position_selected]
            #move_count.pop(position_selected)
            del neighbor_tracking[position_selected]
            #neighbor_tracking.pop(position_selected)
#        else:
            #print 'position select not in layout_old'
       # od = collections.OrderedDict(sorted(layout_old.items()))
       # g.vs['name'] = od.keys()
        
        layout_old, pos = Mover(position_selected, layout_old, illegal_moves)  # move in new direction         
        
        #print 'this is position_selected 2:', position_selected
        #print 'this is pos 1:', pos

        # this will update the count of moves the selected object can perform still
        move_count[pos] = movement_finder(pos, illegal_moves, layout_old, spaceMin, spaceMax)
        new_association_function(pos, layout_old, g, neighbor_tracking, False)
        
        #print 'this is position_selected 3:', position_selected
        #print 'this is pos 2:', pos
        
        #od = collections.OrderedDict(sorted(layout_old.items()))
        #g.vs['name'] = od.keys()
        
        g.vs.find(str(position_selected))['name'] = str(pos)
#        g.vs.find(position_selected)['name'] = pos


        
        if pos != position_selected:
            v1_position = g.vs['name'].index(str(pos))
#            v1_position = g.vs['name'].index(pos)

            if g.degree(v1_position) > 0:
                g.delete_edges( g.incident(str(pos), mode="all"))
#                g.delete_edges( g.incident(pos, mode="all"))
#                g.delete_edges(v1_position) 
        
        #print 'move position_selected =', position_selected
        #print 'move pos =', pos
        
        
    if reaction == 'bind':
        
        position_selected = edge_selector(neighbor_tracking)                  # select a node with neighbors randomly for forming an edge
        
        move_count.pop(position_selected)
        neighbor_tracking.pop(position_selected)
        
        # bind & update move_count and neighbor_tracking dict
        
        new_association_function(position_selected, layout_old, g, neighbor_tracking, True)
        move_count[position_selected] = movement_finder(position_selected, illegal_moves, layout_old, spaceMin, spaceMax)
        
        
        g.simplify(g)                                                        # simplify the graph  
        

    if reaction == 'dissociate':
        
        edge_list = g.get_edgelist()
        edge_selected = random.choice(edge_list)
        g.delete_edges(edge_selected)
    
    
    degree_dist[t] = sum(g.degree())   # Capture degree distribution per step
    degree_dist_all[t] = g.degree()
        
    print reaction
    t += 1

move
move
bind
move
move
bind
move
move
move
move
move
move
move
move
move
move
move
bind
move
move
move
bind
move
move
move
move
move
move
bind
move
move
move
move
move
move
move
move
move
move
move
move
move
move
bind
move
move
move
move
move
move
bind
move
move
move
move
move
bind
move
move
bind
move
bind
bind
move
move
move
move
move
move
move
move
move
bind
move
move
move
move
bind
move
move
move
move
move
move
bind
move
bind
move
move
move
move
move
move
move
bind
bind
move
move
move
move
move
move
move
move
move
move
move
move
move
move
move
move
move
bind
move
move
move
bind
bind
move
move
move
move
move
move
move
bind
move
move
move
bind
move
move
bind
move
bind
move
move
move
bind
bind
bind
bind
move
move
move
bind
move
move
bind
move
move
move
move
move
move
move
move
move
move
move
move
move
dissociate
move
move
move
move
move
bind
move
move
bind
move
move
move
move
bind
move
move
move
move
move
bind
move
move
move
move
move
move
move
bind
move
move
move
move
move
move
move

bind
move
move
dissociate
move
move
move
move
move
bind
move
move
move
bind
bind
bind
move
move
move
bind
dissociate
move
move
move
move
move
move
dissociate
move
move
move
move
move
bind
move
move
move
move
move
move
move
bind
move
move
move
bind
bind
move
move
dissociate
move
move
move
move
bind
move
move
dissociate
move
move
move
move
move
dissociate
dissociate
move
move
dissociate
move
move
move
bind
bind
bind
bind
bind
dissociate
bind
move
bind
bind
move
move
move
move
dissociate
move
dissociate
move
move
bind
move
move
move
dissociate
move
move
bind
move
move
bind
move
dissociate
bind
move
bind
bind
move
move
move
move
move
bind
move
move
bind
dissociate
move
move
bind
move
bind
bind
move
move
bind
move
move
move
move
move
move
move
move
move
move
move
move
move
bind
move
move
move
move
move
move
move
bind
move
move
move
move
bind
move
move
move
bind
move
move
move
dissociate
move
bind
move
bind
bind
bind
move
move
move
move
move
bind
move
move
move
move
move
move
move
move
move


move
bind
move
bind
bind
move
move
move
bind
move
move
move
move
bind
move
move
bind
bind
move
move
move
move
move
bind
bind
bind
bind
move
bind
move
move
move
move
bind
move
move
move
move
move
move
move
dissociate
move
move
dissociate
bind
bind
move
move
dissociate
move
bind
bind
move
bind
move
move
move
move
move
dissociate
dissociate
move
move
move
move
move
move
dissociate
move
move
move
move
move
move
move
bind
move
move
dissociate
move
move
bind
move
move
move
move
move
dissociate
move
move
move
move
move
move
move
bind
move
move
move
move
move
move
move
move
dissociate
move
dissociate
move
move
move
move
move
move
bind
move
move
dissociate
move
move
move
move
bind
dissociate
move
move
bind
bind
bind
move
dissociate
move
move
move
dissociate
move
move
move
move
move
dissociate
move
move
move
move
move
move
move
move
move
move
move
dissociate
move
dissociate
move
move
bind
bind
move
move
move
bind
move
bind
move
bind
move
move
move
move
move
bind
bind
move
move
move
move
move
mov

move
bind
move
bind
dissociate
dissociate
move
move
move
move
dissociate
move
move
dissociate
move
bind
move
move
bind
move
move
dissociate
move
move
move
move
move
bind
move
move
move
move
dissociate
move
move
dissociate
bind
move
move
move
move
dissociate
dissociate
move
bind
dissociate
dissociate
bind
move
bind
bind
bind
move
move
move
move
bind
move
move
dissociate
move
dissociate
dissociate
dissociate
move
move
move
move
move
move
move
bind
move
move
move
move
bind
move
move
move
move
move
move
bind
bind
move
move
move
move
bind
dissociate
move
move
move
move
dissociate
move
move
dissociate
move
bind
move
move
bind
dissociate
bind
dissociate
dissociate
move
move
move
dissociate
bind
move
move
bind
move
dissociate
dissociate
move
move
move
move
move
move
move
move
bind
move
dissociate
move
bind
dissociate
move
move
dissociate
move
dissociate
move
bind
dissociate
move
move
move
move
move
move
move
bind
move
move
move
move
dissociate
dissociate
bind
move
dissociate
move
move
move
mov